In [8]:
import pandas as pd

# Reading data using pandas
df = pd.read_json('customers.json')



'''
Series - 1D Labeled array
DataFrames - 2D Labeled array

A DataFrame is a dictionary of Series under the hood

Both "Series" and "DataFrame" are subclass fo the NDFrame class

NDFrame is the base class in pandas for all data strucutres that handle labeled, array like data

'''
df

,customer_id,name,email,country
0,1,Alice Johnson,alice@example.com,USA
1,2,Bob Smith,bob@example.com,Canada
2,3,Carol Lee,carol@example.com,USA
3,4,David Brown,david@example.com,UK


# Filtering Data using Pandas

- We are trying to look for a SQL "Where" function alternative
- We can do it using df[df['col_name]>x]
- We can also use the .query method

In [6]:
'''
Lets try to find the days where
1. customer id > 2
'''
# Solution 1
df[df['customer_id'] > 2]

'''
Lets try to find the days where
1. customer id > 2
2. name = 'David Brown'
'''
# Solution 1
print("Solution 1")
print(df[(df['customer_id'] > 2) & (df['name'] == 'David Brown')])

# Solution 1
print("Solution 2")
print(df.query('customer_id > 2 and name == "David Brown"'))

Solution 1
   customer_id         name              email country
3            4  David Brown  david@example.com      UK
Solution 2
   customer_id         name              email country
3            4  David Brown  david@example.com      UK


# Joining Data using Pandas

There are 2 ways we can join tables/DataFrames in Pandas

1. merge() -> generally used when joining is based on column_names
2. join() -> generally used when joining is based on index

NOTE - 
1. Merge - When both the columns in tables have null values, they will be matched together

In [12]:
# Reading All the tables

customers = pd.read_json('customers.json')
orders = pd.read_json('orders.json')
categories = pd.read_json('categories.json')
order_items = pd.read_json('order_items.json')
products = pd.read_json('products.json')
vendors = pd.read_json('vendors.json')

display(customers, orders, categories, order_items, products, vendors)

,customer_id,name,email,country
0,1,Alice Johnson,alice@example.com,USA
1,2,Bob Smith,bob@example.com,Canada
2,3,Carol Lee,carol@example.com,USA
3,4,David Brown,david@example.com,UK


,order_id,customer_id,order_date,total
0,101,1,2024-06-01,300
1,102,2,2024-06-03,150
2,103,1,2024-06-10,200
3,104,3,2024-06-15,450


,category_id,category_name
0,301,Accessories
1,302,Displays
2,303,Cameras
3,304,Printers


,order_id,product_id,quantity,unit_price
0,101,201,2,50
1,101,202,1,200
2,102,203,3,50
3,103,204,1,200
4,104,201,3,50
5,104,205,1,300


,product_id,name,category_id,vendor_id
0,201,Keyboard,301,401
1,202,Monitor,302,402
2,203,Mouse,301,401
3,204,Webcam,303,403
4,205,Printer,304,402


,vendor_id,vendor_name
0,401,LogiTech
1,402,Dell
2,403,Microsoft


In [ ]:
'''
merge()
This takes around 13 arguments
1. left table, right table
2. how = "join_name"
3. on = "col_name" - use this when the column name is same
4. left_on , right on = "col_name" - use this when column names are different
5. left_index, right_index = T/F - use index as joining keys
6. sort = T/F - sorts the joining keys lexicographically
7. suffixes = _x, _y - when faced with same column name these suffixes are added

Note - 
1. merge() is a function at the top of the pandas module
   i.e., we can do pd.merge() because of this
2. join() is a method of the Dataframe object
   i.e., we cannot do pd.join(), we will have to provide a dataframe to it, i.e., df.join()
'''

Way_1 = pd.merge(orders, order_items, on = 'order_id').head(1) 
display(Way_1)

Way_2 = pd.merge(orders, order_items, left_on='order_id', right_on = 'order_id').head(1)
display(Way_2)

Way_3 = orders.merge(order_items, on = 'order_id').head(1)
display(Way_3)

,order_id,customer_id,order_date,total,product_id,quantity,unit_price
0,101,1,2024-06-01,300,201,2,50


,order_id,customer_id,order_date,total,product_id,quantity,unit_price
0,101,1,2024-06-01,300,201,2,50


,order_id,customer_id,order_date,total,product_id,quantity,unit_price
0,101,1,2024-06-01,300,201,2,50


In [34]:
# Renaming and updating data types of columns

cus_orders = (customers
            .merge(orders, on = 'customer_id')
            .merge(order_items, on = 'order_id')
            .merge(products, on = 'product_id')
            )\
            .rename(columns={'name_x':'customer_name', 'name_y':'product_name'})\
            .astype({'quantity':'int64', 'unit_price':'int64'})

cus_orders['total_price'] = cus_orders['quantity']*cus_orders['unit_price']

display(cus_orders[['customer_name','product_name','quantity','unit_price', 'total_price']])

# Lets get the total $ amount spent by the customer
display(cus_orders.groupby('customer_name')['total_price'].sum().reset_index())


,customer_name,product_name,quantity,unit_price,total_price
0,Alice Johnson,Keyboard,2,50,100
1,Alice Johnson,Monitor,1,200,200
2,Alice Johnson,Webcam,1,200,200
3,Bob Smith,Mouse,3,50,150
4,Carol Lee,Keyboard,3,50,150
5,Carol Lee,Printer,1,300,300


,customer_name,total_price
0,Alice Johnson,500
1,Bob Smith,150
2,Carol Lee,450
